In [2]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'plants-type-datasets:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1047938%2F7170186%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240212%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240212T134003Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D30a55eb137e406488ce26901207adafb1f252cfff10bc27c093f96d0617fdfc0e72157609f3f10674a26610f3928c1c90905027d3736d42a52c40673e8df5ee577c2b979ebadc3494b74c22e6d161517be2955f2d6c3ec030c54ee0bcc27d5ccbf37902be9d6f196bfdd734c6d2ca0e0b8d8d8adf61d23f05f6a9ea7c3295fed01862d3b8281552bbe0f6303277a4341293d9b6900d8d7aace2392c2ee00c5d1ad48633c4381ce237e116405a41c639ab674808591d486a2ee02e6abd90b9734e4ab81e5e67cd83d124baf6f14f4bdaa49979c4a30d2a10f6425dcaa0fc3db9b5bf833e8459c3ab3dbb28744ea611e0ec2d9e8ce9fc68456a3b3dd67c4574f2c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


The system cannot find the path specified.


[==================================================] 982641259 bytes downloaded
Downloaded and uncompressed: plants-type-datasets
Data source import complete.


In [6]:
import tensorflow as tf

# Check available devices
print("Available devices:", tf.config.list_physical_devices())

# Explicitly specify GPU
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data directories
train_dir = '/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Train_Set_Folder'
test_dir = '/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Test_Set_Folder'
validation_dir = '/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants/Validation_Set_Folder'

# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Create ImageDataGenerator instances with data augmentation for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create image generators for training, testing, and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'
)

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)

# Save the model
model.save('plant_detection_model_2.h5')



Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Found 23972 images belonging to 30 classes.
Found 2998 images belonging to 30 classes.
Found 3030 images belonging to 30 classes.
Epoch 1/50
749/749 [==============================] - 330s 439ms/step - loss: 2.3787 - accuracy: 0.2678 - val_loss: 1.7392 - val_accuracy: 0.4382
Epoch 2/50
749/749 [==============================] - 318s 425ms/step - loss: 1.7779 - accuracy: 0.4387 - val_loss: 1.4852 - val_accuracy: 0.5166
Epoch 3/50
749/749 [==============================] - 214s 285ms/step - loss: 1.5118 - accuracy: 0.5143 - val_loss: 1.2440 - val_accuracy: 0.5894
Epoch 4/50
749/749 [==============================] - 158s 210ms/step - loss: 1.3420 - accuracy: 0.5660 - val_loss: 1.1233 - val_accuracy: 0.6300
Epoch 5/50
749/749 [==============================] - 164s 218ms/step - loss: 1.1969 - accuracy: 0.6112 - val_loss: 1.2038 - val_accur

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('plant_detection_model_2.h5')

# Function to preprocess the image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions
def predict_plant(image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    class_labels = train_generator.class_indices
    predicted_label = list(class_labels.keys())[predicted_class]
    return predicted_label, prediction[0][predicted_class]

# Example usage
input_image_path = 'aloevera42.jpg'
predicted_label, confidence = predict_plant(input_image_path)
print('Predicted plant:', predicted_label)
print('Confidence:', confidence)